In [ ]:
  Importación de librerías necesarias: Importar librerías para manipulación de archivos, datos, visualización y normalización
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import joblib
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
Carga de datos crudos desde archivo CSV: Leer los datos del chancador desde un archivo CSV
file = os.path.join('C:/.../Doc Chancador', 'Chancador_Raw_Data.csv')
df1 = pd.read_csv(file)
df1


In [ ]:
Eliminación de columnas innecesarias: Reducir el dataset a las columnas útiles.
df1 = df1.drop([
	'Corriente.4', 'Unnamed: 0', 'V. Cero.2', 'V. Cero.1',
	'Corriente.3', 'V. Cero', 'Corriente.2', 'Pesómetro', 'Corriente.1'
], axis='columns')
df1

In [ ]:
 Conversión de columnas a tipos apropiados (números y fechas): Preparar los datos para análisis numérico y temporal.
# Convertir columnas a datetime o numérico
columnas_numericas = ['Corriente', 'Presión C. Eje', 'T° Retorno', 'T° socket Linner',
                      'T° Excéntrica', 'T° Alimentación', 'T° Desc. C. Eje', 'Salto Anillo',
                      'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Nivel Taza', 'Setting']

for col in columnas_numericas:
    df1[col] = pd.to_numeric(df1[col], errors='coerce')

df1['Tiempo'] = pd.to_datetime(df1['Tiempo'], errors='coerce')
df1


In [ ]:
Limpieza de filas con datos faltantes: Eliminar registros incompletos.
df1 = df1.dropna()
df1


In [ ]:
 Configuración de 'Tiempo' como índice: Habilitar análisis por tiempo al usar Tiempo como índice.
df1.set_index('Tiempo', inplace=True)
df1


In [ ]:
naAgrupación en ventanas de 12 minUtos (media) : Suavizar los datos agrupando por bloques de 12 minutos.
from functools import partial
from pandas.tseries.frequencies import to_offset

def roundfunc(t, freq):
    freq = to_offset(freq)
    return pd.Timestamp((t.value // freq.delta.value) * freq.delta.value)

df12 = df1.groupby(partial(roundfunc, freq='12T')).median()
df12


In [ ]:
Guardado y recarga de datos agregados: Persistir los datos agrupados y cargarlos con estructura temporal correcta.

file = os.path.join('C:/.../Doc Chancador', '12T')
df12.to_csv(file)
df8 = pd.read_csv(file)
df8['Tiempo'] = pd.to_datetime(df8['Tiempo'], errors='coerce')
df8.set_index('Tiempo', inplace=True)
df8



In [ ]:
Carga de eventos de detención desde Excel: Filtrar solo las detenciones del chancador y clasificar pausas.
file = os.path.join('C:/.../Doc Chancador', 'DetencionesRev.xlsx')
df9 = pd.read_excel(file)
df9 = df9.drop(['Detalle de Reporte','Año','MES','Tipo de Detención (2)',
                'Tiempo Detención (hrs)','Día','Fecha','Unnamed: 0'], axis='columns')
df9 = df9[df9['Equipo'] == '130-CV-004']
df9['Tipo de Detención'].mask(df9['Tipo de Detención'].isin(["Operacional", "Mantención Programada"]),
                              other='Pausa', inplace=True)
df9


In [ ]:
Crear etiquetas de salud del equipo: Asignar etiquetas a cada instante de tiempo según su estado.

col_start = df9['Inicio']
col_end = df9['Fin']
idx = pd.IntervalIndex.from_arrays(col_start, col_end, closed='left')
df9_interval = df9.set_index(idx).drop(columns=['Inicio','Fin'])
df_labels24 = pd.DataFrame(index=df8.index)
df_labels24['Estado de Salud '] = 'Operativo'

for i, r in df9_interval.iterrows():
    mask = (df_labels24.index >= i.left) & (df_labels24.index < i.right)
    df_labels24.loc[mask, 'Estado de Salud '] = r['Tipo de Detención']
df_labels24



In [ ]:
Unir datos agregados con etiquetas: Juntar mediciones con el estado operativo/falla de la máquina.
df25 = pd.concat([df8, df_labels24], axis='columns')
df25


In [ ]:
Codificación binaria de la variable de estado: Eliminar pausas y codificar el estado como binario:

0: Operativo,
1: Falla.
df25 = df25[df25['Estado de Salud '] != 'Pausa']
df25['Estado de Salud '].mask(df25['Estado de Salud '] == 'Operativo', other=0, inplace=True)
df25['Estado de Salud '].mask(df25['Estado de Salud '] == 'Falla', other=1, inplace=True)
df25


In [ ]:
 Separación en variables (X) y etiquetas (Y): Separar entradas del modelo y sus etiquetas.
df28 = df25.drop(columns=['Estado de Salud '])
df29 = df25[['Estado de Salud ']]
df28

In [ ]:
Funciones auxiliares (windowing y split): Construir ventanas deslizantes temporales para entrenamiento y validación.
def datetime_window(dfx, dfy, dfy_total, scaler, size, periods, y_choose='last', ahead=''):
    catcolsx = dfx.select_dtypes(['category']).columns
    dfx[catcolsx] = dfx[catcolsx].apply(lambda x: x.cat.codes)

    catcolsy = dfy.select_dtypes(['category']).columns
    dfy[catcolsy] = dfy[catcolsy].apply(lambda x: x.cat.codes)

    catcolsy_total = dfy_total.select_dtypes(['category']).columns
    dfy_total[catcolsy_total] = dfy_total[catcolsy_total].apply(lambda x: x.cat.codes)

    ind = []
    x = []
    y = []

    def get_windows(win):
        if (win.shape[0] != periods):
            return np.nan

        if (y_choose == 'ahead'):
            wy = dfy.loc[win.index, :]
            try:
                wy = dfy_total.loc[wy.index[-1] + pd.Timedelta(ahead)]
                y.append(wy)
                wx = dfx.loc[win.index, :]
                x.append(wx)
                ind.append(win.index[0])
            except:
                pass
        else:
            wx = dfx.loc[win.index, :]
            x.append(wx)
            wy = dfy.loc[win.index, :]
            wy = wy.iloc[-1] if y_choose == 'last' else wy.iloc[0]
            y.append(wy)
            ind.append(win.index[0])

        return np.nan

    dfx.iloc[:, 0].rolling(size).apply(get_windows)
    return x, y, ind



In [ ]:
Ejecución de separación y normalización: Separar datos en conjuntos de entrenamiento y prueba usando ventanas, con normalización.

def normal_anomal_split(dfx, dfy, size, periods, y_choose='last', ahead='', custom_normal_split=False, normal_test_ratio=0.1, scaler='MinMaxScaler'):
def print_list_shape(lst):
    s = ''
    if (len(lst) > 0):
        s += str(len(lst)) + ' x ' + str(np.array(lst[0]).shape)
    else:
        s += 'empty'
    return s
    x_train, x_test, y_test, x_mix, y_mix, i_mix, myscaler = normal_anomal_split(
    dfx=df28, dfy=df29, size='12T', periods=10, y_choose='last', ahead='',
    custom_normal_split=False, normal_test_ratio=0.1, scaler='MinMaxScaler')

print('Entrenamiento:', print_list_shape(x_train))
print('Test:', print_list_shape(x_test))
print('Etiquetas:', print_list_shape(y_test))

In [ ]:
 Almacenamiento final en estructura para ML: Consolidar todos los datos listos par
a modelamiento (ej. LSTM o Autoencoder).
df28 = {
    'x_train': np.array(x_train),
    'x_test': np.array(x_test),
    'y_test': np.array(y_test),
    'x_mix': np.array(x_mix),
    'y_mix': np.array(y_mix),
    'i_mix': i_mix,
    'myscaler': myscaler,
}